In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
sys.path.append("..")
from scripts.utils import *

In [27]:
df = pd.read_pickle("../data/objective/six-hump-camelback/schemata/01d842b2-f369-4681-b1d3-a29f6f9095a5/03-sampled.pickle")
params = pd.read_csv("../data/configs/01d842b2-f369-4681-b1d3-a29f6f9095a5.csv")

In [3]:
# print("Shape:", df.shape)
print("# rows: \t", df.shape[0])
print("# columns: \t", df.shape[1])

df.head()

# rows: 	 19250
# columns: 	 8


,generation,solution,x,y,fitness,wildcard,defining_length,order
0,0,0000011010101111,-2.858824,0.745098,71.273576,0000*11**01***1*,14,9
1,0,1000101101110100,0.270588,-0.180392,0.107003,******11**1*0***,6,4
2,0,1011001100001100,1.211765,-1.811765,30.174571,*0*10*1*0**01100,14,10
3,0,0101100110100100,-0.905882,0.572549,0.652446,01*110*1**10*100,15,11
4,0,1001101001101011,0.623529,-0.321569,0.685954,1***10*0*******1,15,5


In [6]:
print(df.shape)
print(len(df["wildcard"].unique()))

(19250, 8)
17592


In [7]:
generation_0 = df[df["generation"] == 0]

In [20]:
average_fitness_pop = df.groupby('generation')['fitness'].mean()
average_fitness_pop_df = pd.DataFrame({'generation': average_fitness_pop.index, 'average_fitness': average_fitness_pop.values})


In [28]:
params

,seed,n,Pc,Pm,gmax,nbits,xycutpoint,objective
0,289908,50,1.0,0.01,50,16,8,six-hump camelback


In [53]:
average_fitness_wildcard = df.groupby(['generation', 'wildcard', 'defining_length', 'order'])['fitness'].mean().reset_index()
average_fitness_wildcard
# average_fitness_wildcard_df = pd.DataFrame({'generation': average_fitness.index, 'average_fitness': average_fitness.values})
average_fitness_wildcard['wildcard_count'] = df.groupby(['generation', 'wildcard'])['wildcard'].transform('count')
average_fitness_wildcard

average_fitness_wildcard = average_fitness_wildcard.merge(average_fitness_pop_df[['generation', 'average_fitness']], on=['generation'], suffixes=('_wildcard', '_pop'))

Pc = params["Pc"]
Pm = params["Pm"]
l = params["nbits"]

holland_eq_df = average_fitness_wildcard.copy()
holland_eq_df["rhs_selection"] = holland_eq_df["wildcard_count"] * abs(holland_eq_df["average_fitness"] - -1.0316) / abs(holland_eq_df["fitness"] - -1.0316 + 0.0001) # small term to avoid division by zero
holland_eq_df["rhs_crossover"] = holland_eq_df.apply(lambda x: (Pc * x["defining_length"] / (int(l) - 1)) , axis=1)
holland_eq_df["rhs_mutation"] = holland_eq_df.apply(lambda x: Pm * x["order"], axis=1)
holland_eq_df["rhs_value"] = holland_eq_df.apply(lambda x: x["rhs_selection"] * (1 - x["rhs_crossover"] - x["rhs_mutation"]), axis=1)
holland_eq_df

/var/folders/9c/2nfltbjn5ygb28w9j45bs_4w0000gn/T/ipykernel_18915/1637693790.py:15: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  holland_eq_df["rhs_crossover"] = holland_eq_df.apply(lambda x: (Pc * x["defining_length"] / (int(l) - 1)) , axis=1)


,generation,wildcard,defining_length,order,fitness,wildcard_count,average_fitness,rhs_selection,rhs_crossover,rhs_mutation,rhs_value
0,0,***********01100,4,5,2.330975,1,21.521563,6.706912,0.266667,0.05,4.583057
1,0,*********011*0**,4,4,3.358432,1,21.521563,5.137240,0.266667,0.04,3.561820
2,0,********01***1**,5,3,1.286065,1,21.521563,9.730564,0.333333,0.03,6.195126
3,0,********0111*1**,5,5,0.107003,1,21.521563,19.806003,0.333333,0.05,12.213702
4,0,********1*1*0*11,7,5,3.260073,1,21.521563,5.254976,0.466667,0.05,2.539905
...,...,...,...,...,...,...,...,...,...,...,...
19207,49,100100110*0*01*0,15,13,-0.556230,1,0.087050,2.352725,1.000000,0.13,-0.305854
19208,49,101*****010101**,13,9,0.316553,1,0.087050,0.829704,0.866667,0.09,0.035954
19209,49,101***1***010***,12,7,0.316553,1,0.087050,0.829704,0.800000,0.07,0.107861
19210,49,101*00**0*01****,11,8,0.316553,1,0.087050,0.829704,0.733333,0.08,0.154878


In [45]:
Pc

0    1.0
Name: Pc, dtype: float64

In [17]:
avg_population_fitness = np.average(generation_0["fitness"])
print("Population fitness: ", avg_population_fitness)

for w in generation_0["wildcard"].unique():
    print("Wildcard: ", w, " | defining length: ", np.average(generation_0[generation_0["wildcard"] == w]["defining_length"]), " | order: ", np.average(generation_0[generation_0["wildcard"] == w]["order"]))
    avg_wildcard_schema_fitness = np.average(generation_0[generation_0["wildcard"] == w]["fitness"])
    print("Wildcard fitness: ", avg_wildcard_schema_fitness)
    print("rhs selection:", avg_population_fitness / avg_wildcard_schema_fitness)

Population fitness:  21.52156274638076
Wildcard:  0000*11**01***1*  | defining length:  14.0  | order:  9.0
Wildcard fitness:  71.2735760217718
rhs selection: 0.30195710595195346
Wildcard:  ******11**1*0***  | defining length:  6.0  | order:  4.0
Wildcard fitness:  0.10700340961998595
rhs selection: 201.12969131369616
Wildcard:  *0*10*1*0**01100  | defining length:  14.0  | order:  10.0
Wildcard fitness:  30.174570550607037
rhs selection: 0.7132350967608982
Wildcard:  01*110*1**10*100  | defining length:  15.0  | order:  11.0
Wildcard fitness:  0.6524461833634891
rhs selection: 32.9859585283078
Wildcard:  1***10*0*******1  | defining length:  15.0  | order:  5.0
Wildcard fitness:  0.6859542272975151
rhs selection: 31.374633889450955
Wildcard:  *1*101*1**1*0011  | defining length:  14.0  | order:  10.0
Wildcard fitness:  54.86514437054259
rhs selection: 0.3922629383972935
Wildcard:  ****1*0*****01*0  | defining length:  11.0  | order:  5.0
Wildcard fitness:  0.6524461833634891
rhs selec

In [5]:
for g in range(50):
    print("generation: ", g, " : ", df[df["generation"] == g]["fitness"].max() - df[df["generation"] == g]["fitness"].min())

generation:  0  :  122.69345797068459
generation:  1  :  39.73654446671959
generation:  2  :  54.24859390091626
generation:  3  :  70.4179813346777
generation:  4  :  1.9004959138434094
generation:  5  :  67.03339169747602
generation:  6  :  6.765624067287355
generation:  7  :  92.35696091436719
generation:  8  :  92.54821062204032
generation:  9  :  21.880968443302717
generation:  10  :  6.480054251750975
generation:  11  :  92.35696091436719
generation:  12  :  6.480054251750975
generation:  13  :  92.35696091436719
generation:  14  :  21.880968443302717
generation:  15  :  1.023341839320881
generation:  16  :  71.57735442647504
generation:  17  :  21.880968443302717
generation:  18  :  6.480054251750975
generation:  19  :  1.3409830681762527
generation:  20  :  1.3409830681762527
generation:  21  :  92.35696091436719
generation:  22  :  54.80457145274902
generation:  23  :  6.480054251750975
generation:  24  :  92.35696091436719
generation:  25  :  1.3409830681762527
generation:  26

In [7]:
df[df["wildcard"] == "10011101010001**"]

,generation,solution,x,y,fitness,wildcard
2,0,1001110101000100,0.694118,-0.933333,0.380054,10011101010001**
3669906,1,1001110101000110,0.694118,-0.901961,0.244149,10011101010001**
4652916,1,1001110101000100,0.694118,-0.933333,0.380054,10011101010001**
8716024,2,1001110101000110,0.694118,-0.901961,0.244149,10011101010001**


In [16]:
# Sort by 'avg_fitness' and take the top 10
top_schemas = df.sort_values('avg_fitness',ascending=True).groupby(["generation", 'schema']).head(10)

# Plot 'avg_fitness' for each schema over generations
# for schema in top_schemas.index:
#     df.loc[df['schema'] == schema].plot(y='avg_fitness', label=schema)

# plt.xlabel('Generation')
# plt.ylabel('Average Fitness')
# plt.title('Top 10 Schema Groups Average Fitness Over Generations')
# plt.legend()
# plt.show()

In [22]:
top_schemas.index.names
print(df[df.index.isin([23], level="generation")])

                          sum_fitness  avg_fitness  count
generation schema                                        
23         01110101110*  15479.577223   159.583270     97
           0111010111*0  15479.577223   159.583270     97
           0111010111**  15479.577223   159.583270     97
           011101011*00  15479.577223   159.583270     97
           011101011*0*  15479.577223   159.583270     97
...                               ...          ...    ...
           *****0***1**    528.666667   176.222222      3
           *****0****00    528.666667   176.222222      3
           *****0****0*    528.666667   176.222222      3
           *****0*****0    528.666667   176.222222      3
           *****0******    528.666667   176.222222      3

[6141 rows x 3 columns]
